In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요한 라이브러리 설치
!pip install langchain-community
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install tiktoken
!pip install langchain-google-genai
!pip install chromadb
!pip install langchain_huggingface
!pip install ragas
!pip install faiss-cpu
!pip install faiss-gpu
# !pip install pymilvus  # Milvus

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
import os
import re
import pandas as pd
import bs4
import tiktoken
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output
import time

# 기존 FAISS 대신 Milvus 사용 시 주석 처리된 부분은 그대로
from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
# gemini API
YOUR_API_KEY = ''
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

In [ ]:
# CSV
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250311_semifinal.csv"
df = pd.read_csv(file_path)

In [ ]:
df.columns

Index(['Unnamed: 0', 'ISBN', '분류', '제목', '부제', '저자', '발행자', '발행일', '페이지', '가격',
       '표지', '책소개', '저자소개', '목차', '출판사리뷰', '추천사'],
      dtype='object')

In [ ]:
# 텍스트 분할 (null 값은 빈 문자열 처리)
def split_text(text, chunk_size=1000, overlap=100):
    if text is None or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [ ]:
# 메타데이터 및 핵심 데이터 컬럼 정의
metadata_columns = ['ISBN', '페이지', '가격', '제목', '저자']
vector_doc_columns = ['제목', '분류', '저자','저자소개', '책소개', '목차', '출판사리뷰','추천사']

In [ ]:
# RAG_DB 구성: 각 행의 vector_doc_columns를 하나의 텍스트로 합치고, 메타데이터 별도 저장
RAG_DB = []
for index, row in df.iterrows():
    doc_text = ""
    for col in vector_doc_columns:
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        doc_text += f"{col} : {value}\n"
    chunks = split_text(doc_text)
    metadata = {col: row.get(col, None) for col in metadata_columns}
    for chunk in chunks:
        RAG_DB.append({
            'text': chunk,
            'metadata': metadata
        })

# LangChain Document 생성
documents = [Document(page_content=entry['text'], metadata=entry['metadata']) for entry in RAG_DB]

In [ ]:
# 임베딩 모델 로드 (HuggingFace의 BGE-m3)
hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pickle
from tqdm.notebook import tqdm

In [ ]:
# # 배치 사이즈
# BATCH_SIZE = 17000

# def process_batch(batch_number, documents, hf_embeddings, embedding_dir):
#     start_idx = (batch_number - 1) * BATCH_SIZE
#     end_idx = min(batch_number * BATCH_SIZE, len(documents))
#     if start_idx >= len(documents):
#         print("더 이상 처리할 문서가 없음")
#         return
#     batch_docs = documents[start_idx:end_idx]
#     batch_text_embedding_pairs = []
#     batch_metadata_list = []
#     print(f"Processing batch {batch_number}: documents {start_idx} ~ {end_idx - 1} (총 {len(batch_docs)}개)")
#     for doc in tqdm(batch_docs, desc="Processing Documents in batch", unit="document", leave=False):
#         embedding = hf_embeddings.embed_query(doc.page_content)
#         batch_text_embedding_pairs.append((doc.page_content, embedding))
#         batch_metadata_list.append(doc.metadata)
#     file_path = os.path.join(embedding_dir, f"embedding_results_batch_{batch_number}.pkl")
#     embedding_data = {"text_embedding_pairs": batch_text_embedding_pairs, "metadata_list": batch_metadata_list}
#     with open(file_path, "wb") as f:
#         pickle.dump(embedding_data, f)
#     print(f"Saved batch {batch_number} results to {file_path}")

# # 임베딩 결과 저장 경로 설정 및 배치 처리
# embedding_dir = "/content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/embedding"
# os.makedirs(embedding_dir, exist_ok=True)
# process_batch(10, documents, hf_embeddings, embedding_dir)

# # 모든 pkl 파일 불러와서 병합
# all_text_embedding_pairs = []
# all_metadata_list = []
# for file_name in os.listdir(embedding_dir):
#     if file_name.endswith(".pkl"):
#         file_path = os.path.join(embedding_dir, file_name)
#         with open(file_path, "rb") as f:
#             data = pickle.load(f)
#             all_text_embedding_pairs.extend(data["text_embedding_pairs"])
#             all_metadata_list.extend(data["metadata_list"])
# print(f"불러온 임베딩 문서 수: {len(all_text_embedding_pairs)}")

# merged_embedding_data = {"text_embedding_pairs": all_text_embedding_pairs, "metadata_list": all_metadata_list}
# merged_file_path = os.path.join(embedding_dir, "embedding_results.pkl")
# with open(merged_file_path, "wb") as f:
#     pickle.dump(merged_embedding_data, f)
# print(f"Merged embedding data saved to {merged_file_path}")

In [ ]:
# 벡터 불러오기

embedding_dir = "/content/drive/MyDrive/aiffel_final_project/embedding & Vector Store/embedding"  # 실제 경로로 변경
merged_file_path = os.path.join(embedding_dir, "embedding_results.pkl")

# pkl 파일 불러오기
with open(merged_file_path, "rb") as f:
    merged_embedding_data = pickle.load(f)

# 불러온 데이터에서 임베딩 벡터와 메타데이터 분리
all_text_embedding_pairs = merged_embedding_data["text_embedding_pairs"]
all_metadata_list = merged_embedding_data["metadata_list"]

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_embeddings(
    text_embeddings=all_text_embedding_pairs,
    metadatas=all_metadata_list,
    embedding=hf_embeddings
)

### Retrieval

In [ ]:
# Gemini-1.5-flash LLM 설정
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# FAISS 벡터스토어 retriever 생성 (top k : 5)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# RetrievalQA 체인 구성 (검색 문서도 반환)
dpr_qa_chain = RetrievalQA.from_chain_type(llm=llm_gemini,
                                           retriever=dense_retriever,
                                           return_source_documents=True)

In [ ]:
user_preferences = defaultdict(list)
log_history = []  # 전체 대화 로그

# 기존 멀티턴 프롬프트 (내용은 그대로 사용)
multi_turn_prompt = PromptTemplate(
    input_variables=["history", "query", "impormation", "strategy", "if", "else"],
    template="""
사용자와의 대화 히스토리 :
{history}

사용자의 마지막 질문 :
"{query}"

role :
{impormation}
아래 정보를 바탕으로, 너는 사용자 선호도를 명확히 파악하고, 단호하게 검색 조건을 평가하라.
반드시 아래 기준에 따라 행동해야 한다.

1. 사용자가 책을 찾는 이유를 파악하라.
   - (1) 단순 독서를 원함.
   - (2) 정보나 기술 습득을 위해 찾음.
   - (3) 흥미 또는 취미 생활의 일환으로 찾음.

2. 사용자가 찾고자 하는 책에 대한 사전 지식 수준을 평가하라.
   - (1) 전혀 모름.
   - (2) 카테고리, 작가, 관련 책 이름 등 일부 정보를 보유.
   - (3) 구체적으로 찾고 싶은 책이 있음.
   해당 정보를 간결하게 요약하여 검색 쿼리에 활용해라.

3. 사용자의 취향에 적합한 카테고리를 구체적으로 파악하라.
   아래 항목 중 해당하는 것을 반드시 언급하라.
   - 소설: 현대, 고전, 판타지, SF, 미스터리, 로맨스, 사회 비판, 심리/철학, 역사, 전쟁 소설 등
   - 경제/경영: 기업 경영, 주식, 마케팅, 경제학 입문, 창업, 노동 철학 등
   - 자기계발: 시간관리, 멘탈 관리, 습관 형성, 리더십, 자기 탐색 등
   - 시/에세이: 철학적, 문학적, 감성, 자전적, 여행 시집 등
   - 인문/교양: 철학, 역사, 사회, 심리, 종교/명상 등
   - 취미/실용: 요리, 운동, DIY, 사진, 글쓰기, 음악/예술 등
   - 어린이/청소년: 그림책, 초등 필독서, 청소년 소설, 과학/탐구, 경제/교양 등
   반드시 구체적인 키워드를 포함하라.

{if}
사용자 쿼리를 받았을 때 검색을 할 수 있는 기준

{strategy}
아래 기준에 따라 최종 검색 쿼리를 생성하라:
1. 대화 내용을 종합하여 검색 수행 확률을 0에서 1 사이의 값으로 평가하라. 점수의 증가는 세밀하게 진행해라. (사용자가 원하는 카테고리를 안다면, 0.15점 증가!! 사용자가 책을 원하는 이유를 안다면, 0.2점 증가!! 등)확률이 0.8 이상이면 반드시 검색을 진행하라.
2. 사용자와 멀티챗(사용자 query & LLM 답변)을 10~40 단어로 요약해서 최종 쿼리(query)를 생성해라. 2~5개의 핵심 키워드는 쿼리에 반드시 포함시켜라.
3. 요약문에는 반드시 "책 추천 기준"을 반영하라. 예를 들어, "SF 장르 중 AI 관련 최신 베스트셀러" 또는 "역사 사실에 충실한 근현대사 도서"와 같이 구체적으로 기술하라.
4. 최종 검색 쿼리를 생성할 때, 반드시 추출된 핵심 키워드를 포함하고, 사용자 목적에 맞게 최종 쿼리를 확정하라.
    - 기술 서적을 원한다면, 키워드를 적절하게 나열해라.
    - 소설/자기개발서 등을 원한다면 적절하게 문장 형태로 쿼리를 조합해라.
{else}
현재까지 멀티챗을 종합했을 때, 사용자 선호도를 알 수 없다면, 5번 문항을 명심해라.

5. 만약 사용자 정보가 충분하지 않다면, {impormation}과 {strategy}에 기반하여, 검색 점수를 높일 수 있는 방향으로 적절한 보충 질문을 하나 생성하라.
    - 예시: "관심 있는 도서 카테고리를 명확히 알려줘." 또는 "찾고 있는 책의 목적을 구체적으로 말해줘." 또는 "재미있게 읽었던 책은 뭐야?"
    - 예시: "(카테고리를 모른다.) : 네가 관심있어하는 도서 카테고리가 뭐야?"
    - 예시: "(사용자의 정확한 목적을 모른다.) : 네가 찾고 있는 지식 도서는 초급자 용이야? 상급자 용이야?"

6. 추천 이유 {{reason}}을 작성할때는 책소개,저자,저자소개,출판사리뷰,추천사 등을 적절하게 활용하도록해.
    - 추천 이유에 적합한 내용을 보고 generation 해라.

[출력 형식]
최종 출력은 반드시 아래 형식을 준수하라.
1. 검색 확률: {{score}}
2. 검색 쿼리: "{{final_search_query}}"
3. 추가 질문: "{{follow_up_question}}"
4. 추천 이유: "{{reason}}"
"""
)
search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)

In [ ]:
final_query_prompt = PromptTemplate(
    input_variables=["history", "fallback"],
    template="""
지금까지의 대화 내용을 바탕으로, 사용자의 선호도와 요청을 반영하여 검색에 적절한 최종 쿼리를 생성해.
1. 대화 내용을 요약하고 핵심 키워드를 추출해.
2. 추출된 핵심 키워드를 포함하여, 사용자 목적에 맞는 구체적인 검색 쿼리를 생성해.
대화 내용:
{history}
기본 검색 쿼리: {fallback}
최종 검색 쿼리:
"""
)
final_query_chain = LLMChain(llm=llm_gemini, prompt=final_query_prompt)

In [ ]:
def extract_field(text, field_name):
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

MIN_INFO_LENGTH = 10  # 최소 정보 길이 기준

In [ ]:
#  rewrite & expand
def rewrite_and_expand_query(query, history):
    prompt = f"""Based on the conversation history: '{history}', rewrite and expand the following query to include additional relevant keywords and clarify the user's intent.
Original Query: {query}
Rewritten and Expanded Query:"""
    response = llm_gemini.invoke(prompt)
    rewritten_query = response.text().strip()
    return rewritten_query

In [ ]:
# rerank
def rerank_documents(query, docs):
    # LLM을 크로스 인코더처럼 활용하여 각 문서의 관련도를 평가합니다.
    scored_docs = []
    for doc in docs:
        prompt = f"""Given the query: "{query}", rate the relevance of the following document content on a scale of 0 to 1 (only output a number).
Document Content:
{doc.page_content}
Score:"""
        response = llm_gemini.invoke(prompt)
        try:
            score = float(response.text().strip())
        except Exception as e:
            score = 0.0
        scored_docs.append((score, doc))
    scored_docs.sort(key=lambda x: x[0], reverse=True)
    reranked_docs = [doc for score, doc in scored_docs]
    return reranked_docs

In [ ]:
# summarize
def summarize_document(text):
    prompt = f"""Summarize the following text in a concise manner (around 30-50 words):
{text}
Summary:"""
    response = llm_gemini.invoke(prompt)
    summary = response.text().strip()
    return summary

### Generation

In [ ]:
# ===== 기존의 generate_answer 함수 (문서 통합 후 추천 이유 생성) 수정 =====
def generate_answer(query):
    result = dpr_qa_chain.invoke(query)
    # retrieval 결과에 대해 문서 재정렬 적용
    candidate_docs = result['source_documents']
    reranked_docs = rerank_documents(query, candidate_docs)

    # ISBN 기준으로 문서 aggregation (재정렬된 문서 사용)
    retrieved_isbns = set()
    for doc in reranked_docs:
        isbn = doc.metadata.get("ISBN")
        if isbn:
            retrieved_isbns.add(isbn)
    aggregated_docs = []
    for isbn in retrieved_isbns:
        book_docs = [doc for doc in documents if doc.metadata.get("ISBN") == isbn]
        if not book_docs:
            continue
        aggregated_text = "\n".join([doc.page_content for doc in book_docs])
        # 텍스트가 길면 요약 적용
        if len(aggregated_text) > 500:
            aggregated_text = summarize_document(aggregated_text)
        aggregated_docs.append(Document(page_content=aggregated_text, metadata=book_docs[0].metadata))

    formatted_answers = []
    for doc in aggregated_docs:
        metadata = doc.metadata
        title = metadata.get("제목") or extract_field(doc.page_content, "제목")
        author = metadata.get("저자") or extract_field(doc.page_content, "저자")
        publisher_review = extract_field(doc.page_content, "출판사리뷰")
        book_intro = extract_field(doc.page_content, "책소개")
        if publisher_review and book_intro:
            combined_info = publisher_review + "\n" + book_intro
        elif publisher_review:
            combined_info = publisher_review
        elif book_intro:
            combined_info = book_intro
        else:
            combined_info = ""
        if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
            reason = "추천 이유 정보 없음"
        else:
            # combined_info가 길면 요약 적용
            if len(combined_info) > 300:
                combined_info = summarize_document(combined_info)
            reason_prompt = (
                f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해줘. "
                f"책의 특징이나 강점을 중심으로 설명해주면 좋겠어. 만약 제공된 정보가 충분하지 않다면, '추천 이유 정보 없음'이라고 응답해줘.\n\n정보:\n{combined_info}"
            )
            reason_response = llm_gemini.invoke(reason_prompt)
            generated_reason = reason_response.text().strip()
            if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
                reason = "추천 이유 정보 없음"
            else:
                reason = generated_reason
        formatted = f"{title}\n{author}\n추천 이유: {reason}"
        formatted_answers.append(formatted)
    answer = "\n\n".join(formatted_answers)
    return answer, None

In [ ]:
def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)

In [ ]:
def robust_parse_llm_response(response_text):
    cleaned_text = re.sub(r'\*\*', '', response_text)
    search_score = None
    search_query = None
    follow_up_question = ""
    score_match = re.search(r"검색\s*확률[:：]\s*([\d\.]+)", cleaned_text)
    if score_match:
        try:
            search_score = float(score_match.group(1))
        except Exception as e:
            print("검색 확률 파싱 에러:", e)
    query_match = re.search(r"검색\s*쿼리[:：]\s*(.*)", cleaned_text)
    if query_match:
        search_query = query_match.group(1).strip()
        if search_query.startswith('"') and search_query.endswith('"'):
            search_query = search_query[1:-1].strip()
    follow_match = re.search(r"추가\s*질문[:：]\s*(.*)", cleaned_text)
    if follow_match:
        follow_up_question = follow_match.group(1).strip()
        if follow_up_question in ["(필요 없음)", "(없음)", ""]:
            follow_up_question = ""
        if follow_up_question.startswith('"') and follow_up_question.endswith('"'):
            follow_up_question = follow_up_question[1:-1].strip()
    return search_score, search_query, follow_up_question

In [ ]:
def search_and_generate_answer(query, query_history):
    # 쿼리 재작성/확장을 적용
    rewritten_query = rewrite_and_expand_query(query, "\n".join(query_history[-5:]))
    print(f"\n[🔄 재작성/확장된 쿼리]: {rewritten_query}")
    query = rewritten_query

    while True:
        query_summary = "\n".join(query_history[-5:])
        search_decision_dict = search_query_chain.invoke({
            "history": query_summary,
            "query": query,
            "if": "✅ 검색이 가능한 경우:",
            "else": "❌ 아직 검색이 불가능한 경우:",
            "impormation": "📌 사용자 선호도 분석:",
            "strategy": "🔍 검색 전략:",
        })
        response_text = search_decision_dict["text"].strip()
        print("\n[🔍 LLM 응답 확인]\n", response_text)
        search_score, base_search_query, follow_up_question = robust_parse_llm_response(response_text)
        print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")
        if search_score is None:
            print("\n[❌ LLM 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue
        if search_score >= 0.8 and base_search_query:
            final_search_query = final_query_chain.invoke({
                "history": "\n".join(query_history),
                "fallback": base_search_query
            })["text"].strip()
            print(f"\n[🔎 최종 검색 쿼리 생성]: {final_search_query}")
            answer, sources = generate_answer(final_search_query)
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{answer}\n\n[📚 책 정보]\n{book_info}"
                print("\n[📚 책 정보]\n", book_info)
            else:
                answer_with_info = answer
            return answer_with_info
        if follow_up_question:
            print(f"\n[🤖 보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[📚 사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue
        if search_score < 0.8 or not base_search_query:
            print("\n[❌ 검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [ ]:
def interactive_multi_turn_qa():
    query_history = []
    while True:
        clear_output(wait=True)
        print("📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)
        query = input("질문을 입력하세요: ")
        if query.lower() == 'quit':
            print("\n[📝 대화 저장 중...]")
            log_history.append(query_history)
            print("대화 저장 완료")
            break
        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)
        print("\n[💬 AI의 답변]")
        print(answer)
        query_history.append(f"AI: {answer}")
        input("\n-> 계속하려면 Enter를 누르세요...")

def show_log_history():
    print("\n[ 전체 대화 로그]")
    for i, session in enumerate(log_history, 1):
        print(f"\n 대화 세션 {i}:\n")
        print("\n".join(session))
        print("-" * 50)

In [ ]:
# 실행
interactive_multi_turn_qa()

📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)
--------------------------------------------------
질문을 입력하세요: 역사책을 찾고 있어

[🔄 재작성/확장된 쿼리]: Several options, depending on what the user might want:

**Option 1 (Broad Search):**

한국 역사책을 찾고 있어요.  구체적으로는 [시대] 시대 또는 [주제] 에 대한 책을 찾고 있습니다.  [저자]의 책도 괜찮고, [출판사]에서 나온 책도 좋습니다.  학술적인 책보다는 [대중서/전문서/어린이책]을 선호합니다.


* **Translation:** I'm looking for a history book. Specifically, I'm looking for a book about the [era] period or on the topic of [subject]. A book by [author] would be good, or one published by [publisher] is also fine. I prefer [popular book/academic book/children's book] over [the opposite].

**Option 2 (Specific Search):**

[특정 국가]의 [특정 시대] 역사에 대한 책을 찾고 있어요.  [특정 사건]이나 [특정 인물]에 대해 자세히 다룬 책이면 좋겠습니다.  [학술적/대중적]인 서술 방식을 선호합니다.


* **Translation:** I'm looking for a book about the history of [specific country] during the [specific era].  A book that details [specific event] or [specific person] would be ideal. I prefer a [scholarly/popular] writing


[💬 AI의 답변]
한눈에 쏙쏙 들어오는 한국사 달달노트
최우승 지음
추천 이유: 추천 이유 정보 없음

조선왕조실록1
이성무 지음
추천 이유: 추천 이유 정보 없음

토닥토닥 쓸모 있는 초등 어린이 한국사
박훈 글 지음
추천 이유: 추천 이유 정보 없음

상위 1% 청소년을 위한 조선왕조실록 5
김흥중 지음
추천 이유: 추천 이유 정보 없음

조선 왕실 이야기
공준원 지음
추천 이유: 추천 이유 정보 없음


KeyboardInterrupt: Interrupted by user

In [ ]:
# def initialize_documents():
#     # CSV 파일 로드
#     file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250311_semifinal.csv"
#     df = pd.read_csv(file_path)

#     # RAG_DB 구성: 각 행의 vector_doc_columns를 하나의 텍스트로 합치고, 메타데이터 별도 저장
#     RAG_DB = []
#     for index, row in df.iterrows():
#         doc_text = ""
#         for col in vector_doc_columns:
#             value = row.get(col, "")
#             if pd.isnull(value):
#                 value = ""
#             doc_text += f"{col} : {value}\n"
#         chunks = split_text(doc_text)
#         metadata = {col: row.get(col, None) for col in metadata_columns}
#         for chunk in chunks:
#             RAG_DB.append({
#                 'text': chunk,
#                 'metadata': metadata
#             })
#     # LangChain Document 생성
#     documents = [Document(page_content=entry['text'], metadata=entry['metadata']) for entry in RAG_DB]
#     return documents

# def interactive_multi_turn_qa():
#     # 매 실행 시 문서, 대화 히스토리, 사용자 선호도 등을 내부에서 새로 정의합니다.
#     documents = initialize_documents()
#     query_history = []  # 대화 기록 초기화
#     user_preferences = defaultdict(list)

#     while True:
#         clear_output(wait=True)
#         print("📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
#         print("-" * 50)

#         query = input("질문을 입력하세요: ")
#         if query.lower() == 'quit':
#             print("\n[📝 대화 저장 중...]")
#             # 로그 저장 등 필요한 작업 수행
#             print("대화 저장 완료")
#             break

#         query_history.append(f"사용자: {query}")
#         answer = search_and_generate_answer(query, query_history)

#         print("\n[💬 AI의 답변]")
#         print(answer)
#         query_history.append(f"AI: {answer}")

#         input("\n-> 계속하려면 Enter를 누르세요...")

# # 최종 실행
# interactive_multi_turn_qa()